# Chess Dataset

In [1]:
import re
import numpy as np
import pandas as pd
import json as js
import chessdotcom as chess
from datetime import datetime

## Chess.com API

In [2]:
def get_player_profile(user):
    df_player = pd.DataFrame({user:chess.get_player_profile(user).json}).T
    return df_player

In [3]:
def get_player_stats(user): 
    df_stats = pd.json_normalize(chess.get_player_stats(user).json, sep='_')
    df_stats.index = [user]
    return df_stats

In [4]:
def get_country_name(country):
    return chess.get_country_details(country).json['name']

In [5]:
def get_country_players(country):
    return chess.get_country_players(country).json['players']

In [6]:
def get_player_games_by_month(user, year, month):
    return str(chess.get_player_games_by_month_pgn(user, year, month)).split('Event')[1:]

In [7]:
def get_titled_players(title):
    return chess.get_titled_players(title).json['players']

In [8]:
def get_club_members(club): 
    return chess.get_club_members(club).json['all_time']

## Player Record

In [9]:
def get_timestamp(timef):
    return datetime.fromtimestamp(timef).strftime('%Y-%m-%d %I:%M:%S %p')

In [10]:
def _clean_record(record):
    record['country'] = record['country'].apply(lambda x: x.split('/')[-1])
    record.insert(2,'country_name', record['country'].apply(lambda x: get_country_name(x)))
    cols_time = ['joined', 'last_online']
#     record[cols_time] = record[cols_time].apply(lambda x: get_timestamp(x))
    for col in cols_time: record[col] = record[col].apply(lambda x: get_timestamp(x))
    record = record.infer_objects()
    return record

In [11]:
def get_player_record(user,getstats=True):
    record = pd.DataFrame()
    try:
        profile = get_player_profile(user) 
        if getstats:
            stats = get_player_stats(user) 
            record = _clean_record(pd.concat([profile, stats], axis=1, sort=False))
        else: record = _clean_record(profile)
    except Exception as details:
        print ('>> ERROR: unable to get %s player record \n>> %s'%(user,details))
    return record

In [12]:
def get_multiple_player_record(players,getstats=True):
    players_record = []
    for player in players:
        try: 
            player_n = get_player_record(player,getstats)
            if not player_n.empty: players_record.append(player_n)
        except Exception as details:
            print ('>> ERROR: unable to get %s record>> %s'%(player,details))
    return pd.concat(players_record) 

## Player Games 

In [13]:
def _clean_game_info(df_game):
    colname = ['Date','White','Black','Result','ECO','Opening','WhiteElo','BlackElo','TimeControl']
    rename_cols = dict(zip(range(len(colname)),colname))
    df_game = df_game.rename(columns=rename_cols) 
    for col in colname: 
        if col in ['Opening','TimeControl']: df_game[col]=df_game[col].apply(lambda x: x.replace('openings/','').replace('TimeControl "','').split('Variation')[0])
        else: df_game[col]=df_game[col].apply(lambda x: re.findall(r'"(.*?)"',x)[0])
    df_game = df_game.sort_values(by='Date')
    return df_game
    

In [14]:
def get_game_info(game): 
    game_info = "\[Date \W\S+]|\[White \W\S+]|\[WhiteElo \W\S+]|\[Black \W\S+]|\[BlackElo \W\S+]|\[Result \W\S+]|\[ECO \W\S+]|TimeControl \D+\w+|openings/\D+"
    data = re.findall(game_info,str(game))
#     print(data)
    df = pd.DataFrame(data).T
    return df

In [15]:
def get_player_games_montly(user, year, month):
    df_all = pd.DataFrame()
    try:
        monthly_games = get_player_games_by_month(user, year, month)
        if monthly_games: df_all = _clean_game_info(pd.concat([get_game_info(game) for game in monthly_games]))
    except Exception as details:
        print ('>> ERROR: unable to get %s games for %s-%s>> %s'%(user,year,month,details))
    return df_all

In [16]:
def get_player_games_yearly(user, year):
    months = []
    for month in range(1,13):
        df_month = get_player_games_montly(user, year, month) 
        if not df_month.empty: months.append(df_month)
    df_year = pd.concat(months)
    df_year.insert(0,'user',user)
    return df_year

## Titled Players  

In [17]:
def get_titled_players_record(title):
    print('Getting %s players ...' % title)
    player_title = get_titled_players(title)
    return get_multiple_player_record(player_title)

In [18]:
def get_all_titled_players():
    titles = ['WGM','WIM','WFM','WCM','WNM','GM','IM','FM','CM','NM'] 
    df_all_titled = pd.concat([get_titled_players_record(title) for title in titles])
    return df_all_titled 

In [19]:
def get_titled_players_games(players, year=2020):
    all_games = []
    for n_idx, user in enumerate(players):
        print('\tuser(%d):%s'%(n_idx,user))
        try:
            df_year = get_player_games_yearly(user, year) 
            if not df_year.empty: all_games.append(df_year)
            if (n_idx>0) and n_idx%500==0:
                export_df(pd.concat(all_games),'./chess_datasets/titled_games/%d_chesscom_titled_players_games'%n_idx,idx=False)
        except Exception as details:
            print ('>> ERROR: user=%d unable to get %s games for %d>> %s'%(n_idx, user,year,details))
    return pd.concat(all_games)

## Club Members

In [20]:
def get_club_members_record(club):
    print('Getting %s members ...' % club)
    members = [memb['username'] for memb in get_club_members(club)]
    return get_multiple_player_record(members)

## Worldwide Players @ Chess.com

In [21]:
def get_worldwide_players(ncountries): 
    country_all, country_name, nplayers = [], [], []
    
    for country in ncountries:
        try: 
            num_plys = len(get_country_players(country))
            if num_plys:
                nplayers.append(num_plys)
                country_name.append(get_country_name(country)) 
                country_all.append(country)
        except Exception as details: 
            ncountries.pop(ncountries.index(country))
            print ('>> ERROR: unable to get %s players\n>> %s'%(country, details))
            
    df_ct = pd.DataFrame({'country': country_all, 'country_name': country_name, 'nPlayers': nplayers})
    df_ct = df_ct.sort_values(by='nPlayers', ascending=False).reset_index(drop=True)
    return df_ct

## Export to csv

In [22]:
def export_df(df, outname, idx=True):
    date = datetime.today().strftime("%d_%m_%Y") 
    df.to_csv('%s_%s.csv'%(outname, date), index=idx)

### Getting Titled Payers

In [23]:
pd.options.display.max_columns = None

In [24]:
# titled_players = get_all_titled_players()
# export_df(titled_players, './chess_datasets/chesscom_titled_players')
# titled_players.shape

In [25]:
# WGMs = get_titled_players_record('WGM')
# export_df(WGMs,'./chess_datasets/WGMs_titled_players')

### Getting Club Members

In [26]:
# chesskid = get_club_members_record('chesskid-com-official-club')
# export_df(chesskid,'./chess_datasets/chesskid_players')

In [27]:
# chess_us = get_club_members_record('uschess')
# export_df(chess_us,'./chess_datasets/chess_us')

In [28]:
# chess_university = get_club_members_record('chess-university')
# export_df(chess_university,'./chess_datasets/chess_university') 

In [29]:
# chess_university.shape #(85973, 86)

### Getting Worlwide Players 

In [30]:
# df_c = pd.read_csv('./chess_datasets/chesscom_university_08_12_2020.csv', index_col=0)
# chess_countries = df_c['country'].dropna().unique().tolist() 

In [31]:
# chesscom_players = get_worldwide_players(chess_countries)
# export_df(chesscom_players,'./chess_datasets/chesscom_countries_players')

In [32]:
# chesscom_players.head(n=10)

In [33]:
# print ('Total Players=%d from %d countries' % (chesscom_players['nPlayers'].sum(), chesscom_players.shape[0]))

In [34]:
# usa_players = get_country_players('US') #Error getting US players
# usa_players

### Getting Player Games

In [35]:
df_title = pd.read_csv('./chess_datasets/chesscom_titled_players_08_12_2020.csv', index_col=0)

In [36]:
# Keep players without name
df_nan = df_title[df_title['name'].isin([np.nan,np.NAN,np.NaN, 'nan','NaN'])]

In [37]:
# Remove duplicate players
df_title.drop_duplicates(subset=['name'],keep='last',inplace=True)
#include nan players
df_title = df_title.append(df_nan)
df_title.shape

(8935, 86)

In [38]:
df_title.head()

,@id,avatar,country_name,country,followers,is_streamer,joined,last_online,location,name,player_id,status,title,url,username,fide,chess_rapid_last_rating,chess_rapid_last_date,chess_rapid_last_rd,chess_rapid_record_win,chess_rapid_record_loss,chess_rapid_record_draw,chess_daily_last_rating,chess_daily_last_date,chess_daily_last_rd,chess_daily_record_win,chess_daily_record_loss,chess_daily_record_draw,chess_daily_record_time_per_move,chess_daily_record_timeout_percent,chess_rapid_best_rating,chess_rapid_best_date,chess_rapid_best_game,chess_bullet_last_rating,chess_bullet_last_date,chess_bullet_last_rd,chess_bullet_best_rating,chess_bullet_best_date,chess_bullet_best_game,chess_bullet_record_win,chess_bullet_record_loss,chess_bullet_record_draw,chess_blitz_last_rating,chess_blitz_last_date,chess_blitz_last_rd,chess_blitz_best_rating,chess_blitz_best_date,chess_blitz_best_game,chess_blitz_record_win,chess_blitz_record_loss,chess_blitz_record_draw,tactics_highest_rating,tactics_highest_date,tactics_lowest_rating,tactics_lowest_date,lessons_highest_rating,lessons_highest_date,lessons_lowest_rating,lessons_lowest_date,puzzle_rush_best_total_attempts,puzzle_rush_best_score,chess_daily_best_rating,chess_daily_best_date,chess_daily_best_game,puzzle_rush_daily_total_attempts,puzzle_rush_daily_score,twitch_url,chess960_daily_last_rating,chess960_daily_last_date,chess960_daily_last_rd,chess960_daily_best_rating,chess960_daily_best_date,chess960_daily_best_game,chess960_daily_record_win,chess960_daily_record_loss,chess960_daily_record_draw,chess960_daily_record_time_per_move,chess960_daily_record_timeout_percent,chess_daily_tournament_points,chess_daily_tournament_withdraw,chess_daily_tournament_count,chess_daily_tournament_highest_finish,chess960_daily_tournament_points,chess960_daily_tournament_withdraw,chess960_daily_tournament_count,chess960_daily_tournament_highest_finish
ahachess,https://api.chess.com/pub/player/ahachess,https://images.chesscomfiles.com/uploads/v1/us...,Vietnam,VN,30,False,2011-06-13 10:01:18 PM,2020-11-16 02:33:06 PM,Tp. Hồ Chí Minh,An Nguyen Thi Thanh,5274038,premium,WGM,https://www.chess.com/member/ahachess,ahachess,0.0,NaN,NaN,NaN,NaN,NaN,NaN,991.0,1.528405e+09,216.0,3.0,0.0,0.0,48738.0,0.0,NaN,NaN,NaN,2255.0,1.593953e+09,50.0,2493.0,1.534284e+09,https://www.chess.com/live/game/3006554673,404.0,277.0,26.0,2317.0,1.599636e+09,43.0,2424.0,1.534111e+09,https://www.chess.com/live/game/4726301080,266.0,217.0,50.0,2769.0,1.588841e+09,1372.0,1.308145e+09,2351.0,1.537289e+09,897.0,1.525613e+09,33.0,30.0,1200.0,1.308028e+09,https://www.chess.com/game/daily/194095878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ambotsari,https://api.chess.com/pub/player/ambotsari,https://images.chesscomfiles.com/uploads/v1/us...,Greece,GR,3,False,2020-07-13 09:51:45 PM,2020-11-16 10:51:20 AM,NaN,Anna-Maria Botsari,86647612,premium,WGM,https://www.chess.com/member/ambotsari,ambotsari,0.0,2183.0,1.598540e+09,139.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2324.0,1.597405e+09,https://www.chess.com/live/game/5330488722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
anastasiyakarlovych,https://api.chess.com/pub/player/anastasiyakar...,https://images.chesscomfiles.com/uploads/v1/us...,Greece,GR,66,False,2017-05-05 03:22:49 PM,2020-12-08 02:29:19 PM,NaN,Anastasiya Karlovych,35309738,premium,WGM,https://www.chess.com/member/anastasiyakarlovych,anastasiyakarlovych,NaN,1895.0,1.591051e+09,280.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1895.0,1.591051e+09,https://www.chess.com/live/game/4939748822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1966.0,1.605608e+09,90.0,2451.0,1.590358e+09,https://www.chess.com/live/game/4943059733,36.0,28.0,5.0,2322.0,1.595958e+09,400.0,1.552649e+09,NaN,NaN,NaN,NaN,34.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [39]:
titled_users = df_title.index.tolist()
len(titled_users)

8935

In [40]:
df_titled_games = get_titled_players_games(titled_users)

	user(0):ahachess
>> ERROR: unable to get ahachess games for 2020-4>> list index out of range
>> ERROR: unable to get ahachess games for 2020-9>> list index out of range
	user(1):ambotsari
	user(2):anastasiyakarlovych
>> ERROR: unable to get anastasiyakarlovych games for 2020-6>> list index out of range
	user(3):ankudinova_yelena
	user(4):anne93
	user(5):armqueen
	user(6):ati1517
>> ERROR: unable to get ati1517 games for 2020-5>> list index out of range
>> ERROR: unable to get ati1517 games for 2020-8>> list index out of range
	user(7):atousa1
	user(8):awarak
>> ERROR: unable to get awarak games for 2020-10>> list index out of range
>> ERROR: unable to get awarak games for 2020-11>> list index out of range
	user(9):axvesik
	user(10):backlight26
>> ERROR: unable to get backlight26 games for 2020-7>> list index out of range
>> ERROR: unable to get backlight26 games for 2020-9>> list index out of range
	user(11):bairakovanova
	user(12):belakovskaia-arz
>> ERROR: user=12 unable to get bela

>> ERROR: unable to get kiranmanisha games for 2020-9>> list index out of range
	user(65):klavdia_nik
>> ERROR: unable to get klavdia_nik games for 2020-5>> list index out of range
>> ERROR: unable to get klavdia_nik games for 2020-10>> list index out of range
>> ERROR: unable to get klavdia_nik games for 2020-11>> list index out of range
	user(66):kopeisk81
	user(67):krapinek9
	user(68):ktsatsalashvili
>> ERROR: unable to get ktsatsalashvili games for 2020-11>> list index out of range
>> ERROR: unable to get ktsatsalashvili games for 2020-12>> list index out of range
	user(69):lanita_1408
>> ERROR: unable to get lanita_1408 games for 2020-8>> list index out of range
	user(70):lenkaptacnikova
>> ERROR: unable to get lenkaptacnikova games for 2020-4>> list index out of range
>> ERROR: unable to get lenkaptacnikova games for 2020-8>> list index out of range
	user(71):lexiel
>> ERROR: unable to get lexiel games for 2020-5>> list index out of range
	user(72):lisychess
	user(73):liverpoolbo

>> ERROR: unable to get agamat games for 2020-11>> list index out of range
	user(160):akromion2712
>> ERROR: unable to get akromion2712 games for 2020-7>> list index out of range
>> ERROR: unable to get akromion2712 games for 2020-9>> list index out of range
	user(161):alegro
>> ERROR: unable to get alegro games for 2020-3>> list index out of range
>> ERROR: unable to get alegro games for 2020-4>> list index out of range
	user(162):aleksandramilovic
	user(163):alekschess100
>> ERROR: unable to get alekschess100 games for 2020-4>> list index out of range
>> ERROR: unable to get alekschess100 games for 2020-9>> list index out of range
	user(164):alfilescruzados
	user(165):alindsay211
>> ERROR: unable to get alindsay211 games for 2020-2>> list index out of range
>> ERROR: unable to get alindsay211 games for 2020-12>> list index out of range
	user(166):ama18
>> ERROR: unable to get ama18 games for 2020-4>> list index out of range
>> ERROR: unable to get ama18 games for 2020-7>> list index 

>> ERROR: unable to get esp21caxita games for 2020-10>> list index out of range
>> ERROR: unable to get esp21caxita games for 2020-11>> list index out of range
	user(239):ethylalcohol
>> ERROR: unable to get ethylalcohol games for 2020-3>> list index out of range
>> ERROR: unable to get ethylalcohol games for 2020-5>> list index out of range
>> ERROR: unable to get ethylalcohol games for 2020-8>> list index out of range
	user(240):evgenia89
	user(241):evi2703
	user(242):excitingtrap
	user(243):falcon28dz
	user(244):feketebarika
>> ERROR: unable to get feketebarika games for 2020-4>> list index out of range
>> ERROR: unable to get feketebarika games for 2020-5>> list index out of range
	user(245):fiona
	user(246):froggy64
>> ERROR: unable to get froggy64 games for 2020-5>> list index out of range
	user(247):gabkos
>> ERROR: unable to get gabkos games for 2020-5>> list index out of range
	user(248):gettingmymojoback
>> ERROR: user=248 unable to get gettingmymojoback games for 2020>> No o

>> ERROR: unable to get nona_d games for 2020-12>> list index out of range
	user(328):nw_tynka
	user(329):n_0704
>> ERROR: unable to get n_0704 games for 2020-7>> list index out of range
	user(330):oiam03
	user(331):olgavasiliev
	user(332):olga_zhuravleva
>> ERROR: unable to get olga_zhuravleva games for 2020-4>> list index out of range
	user(333):ortizanahi2001
>> ERROR: unable to get ortizanahi2001 games for 2020-7>> list index out of range
	user(334):pandasforlife
>> ERROR: unable to get pandasforlife games for 2020-4>> list index out of range
	user(335):parmigiano
	user(336):parnali1611
>> ERROR: unable to get parnali1611 games for 2020-9>> list index out of range
	user(337):patty43
>> ERROR: user=337 unable to get patty43 games for 2020>> No objects to concatenate
	user(338):perezjennifer1987
>> ERROR: unable to get perezjennifer1987 games for 2020-6>> list index out of range
	user(339):petroniy_arbitr
	user(340):pinoe
>> ERROR: user=340 unable to get pinoe games for 2020>> No obj

...

	user(8652):chesswooog
>> ERROR: unable to get chesswooog games for 2020-2>> list index out of range
>> ERROR: unable to get chesswooog games for 2020-9>> list index out of range
>> ERROR: unable to get chesswooog games for 2020-10>> list index out of range
>> ERROR: unable to get chesswooog games for 2020-12>> list index out of range
	user(8653):chessyinstinct
>> ERROR: unable to get chessyinstinct games for 2020-1>> list index out of range
	user(8654):chess_dog06
>> ERROR: unable to get chess_dog06 games for 2020-8>> list index out of range
	user(8655):chess_foreveryone
	user(8656):chess_international
	user(8657):chess_player_17
>> ERROR: unable to get chess_player_17 games for 2020-1>> list index out of range
>> ERROR: unable to get chess_player_17 games for 2020-2>> list index out of range
>> ERROR: unable to get chess_player_17 games for 2020-3>> list index out of range
>> ERROR: unable to get chess_player_17 games for 2020-4>> list index out of range
>> ERROR: unable to get chess

	user(8709):frankggomez
	user(8710):frenchandslav
	user(8711):frequentblunders
	user(8712):frostdemonstorm
>> ERROR: unable to get frostdemonstorm games for 2020-3>> list index out of range
>> ERROR: unable to get frostdemonstorm games for 2020-12>> list index out of range
	user(8713):fumitoks
	user(8714):fytnob
>> ERROR: unable to get fytnob games for 2020-4>> list index out of range
	user(8715):g2_g3
	user(8716):galatians522
	user(8717):gawk
	user(8718):giftedcreation
>> ERROR: unable to get giftedcreation games for 2020-3>> list index out of range
	user(8719):giraffe1912
	user(8720):greatdamnedredspot
	user(8721):guyford
	user(8722):gympod
>> ERROR: unable to get gympod games for 2020-2>> list index out of range
	user(8723):gyoung11
>> ERROR: unable to get gyoung11 games for 2020-11>> list index out of range
	user(8724):h4chess
>> ERROR: user=8724 unable to get h4chess games for 2020>> No objects to concatenate
	user(8725):hanoirocks
>> ERROR: user=8725 unable to get hanoirocks game

>> ERROR: unable to get mc4chess games for 2020-5>> list index out of range
	user(8781):meaniebub
	user(8782):meekinskris
>> ERROR: unable to get meekinskris games for 2020-10>> list index out of range
>> ERROR: unable to get meekinskris games for 2020-12>> list index out of range
	user(8783):mfcampeao
	user(8784):mightygmpretender
>> ERROR: unable to get mightygmpretender games for 2020-5>> list index out of range
	user(8785):mitfit64
>> ERROR: unable to get mitfit64 games for 2020-12>> list index out of range
	user(8786):morphinho22
	user(8787):mortalbovine
>> ERROR: unable to get mortalbovine games for 2020-2>> list index out of range
	user(8788):mr-leahy
>> ERROR: unable to get mr-leahy games for 2020-5>> list index out of range
>> ERROR: unable to get mr-leahy games for 2020-6>> list index out of range
>> ERROR: unable to get mr-leahy games for 2020-7>> list index out of range
>> ERROR: unable to get mr-leahy games for 2020-9>> list index out of range
>> ERROR: unable to get mr-le

>> ERROR: unable to get ru_b9 games for 2020-10>> list index out of range
	user(8843):rwhaines
>> ERROR: user=8843 unable to get rwhaines games for 2020>> No objects to concatenate
	user(8844):saccona_silvano_m
>> ERROR: unable to get saccona_silvano_m games for 2020-6>> list index out of range
	user(8845):sailorknight
	user(8846):saisree
>> ERROR: unable to get saisree games for 2020-3>> list index out of range
>> ERROR: unable to get saisree games for 2020-4>> list index out of range
>> ERROR: unable to get saisree games for 2020-8>> list index out of range
	user(8847):scgamecock
	user(8848):schwarzschidl
>> ERROR: unable to get schwarzschidl games for 2020-11>> list index out of range
	user(8849):scott-fox21
>> ERROR: unable to get scott-fox21 games for 2020-1>> list index out of range
>> ERROR: unable to get scott-fox21 games for 2020-4>> list index out of range
>> ERROR: unable to get scott-fox21 games for 2020-5>> list index out of range
>> ERROR: unable to get scott-fox21 games 

>> ERROR: unable to get yy2324 games for 2020-3>> list index out of range
	user(8925):zaidejaschegis
>> ERROR: unable to get zaidejaschegis games for 2020-2>> list index out of range
>> ERROR: unable to get zaidejaschegis games for 2020-4>> list index out of range
>> ERROR: unable to get zaidejaschegis games for 2020-5>> list index out of range
>> ERROR: unable to get zaidejaschegis games for 2020-6>> list index out of range
>> ERROR: unable to get zaidejaschegis games for 2020-7>> list index out of range
>> ERROR: unable to get zaidejaschegis games for 2020-8>> list index out of range
>> ERROR: unable to get zaidejaschegis games for 2020-10>> list index out of range
>> ERROR: unable to get zaidejaschegis games for 2020-11>> list index out of range
>> ERROR: unable to get zaidejaschegis games for 2020-12>> list index out of range
	user(8926):zane33
	user(8927):zbbesque
>> ERROR: user=8927 unable to get zbbesque games for 2020>> No objects to concatenate
	user(8928):zeitnotakrobat
	user

In [41]:
df_titled_games.head()

,user,Date,White,Black,Result,ECO,Opening,WhiteElo,BlackElo,TimeControl
0,ahachess,2020.01.07,ahachess,quesfr,1-0,D30,"Queens-Gambit-Declined""]\n[UTCDate """,2252,2200,180
0,ahachess,2020.01.07,Olga_Zhuravleva,ahachess,0-1,B34,Sicilian-Defense-Open-Accelerated-Dragon-,2322,2231,180
0,ahachess,2020.01.07,FearNoEvil12,ahachess,1-0,B23,Sicilian-Defense-Closed-Grand-Prix-,2054,2199,180
0,ahachess,2020.01.07,ahachess,Bagirova,1/2-1/2,E00,"Catalan-Opening""]\n[UTCDate """,2237,2123,180
0,ahachess,2020.01.07,r31415,ahachess,1-0,B06,Modern-Defense-with-,2471,2246,180


In [42]:
export_df(df_titled_games,'./chess_datasets/chesscom_titled_players_games',idx=False)

In [43]:
df_titled_games.shape

(5422620, 10)

### Parcerita1 :)

In [44]:
nk = get_player_record('parcerita1')
nk.T

,parcerita1
@id,https://api.chess.com/pub/player/parcerita1
avatar,https://images.chesscomfiles.com/uploads/v1/us...
country_name,Colombia
country,CO
followers,48
is_streamer,False
joined,2014-06-03 12:18:44 PM
last_online,2020-12-11 09:16:33 PM
location,California
name,Nadya Ortiz


In [45]:
df_nk = get_player_games_yearly('parcerita1', 2020)
df_nk

,user,Date,White,Black,Result,ECO,Opening,WhiteElo,BlackElo,TimeControl
0,parcerita1,2020.01.01,parcerita1,YoungMegaMind,0-1,B01,Scandinavian-Defense-Mieses-Kotrc-Gubinsky-Mel...,2109,2184,180
0,parcerita1,2020.01.01,el_sporco,parcerita1,0-1,B41,"Sicilian-Defense-Kan-Maroczy-Bind-Formation""]\...",2268,2116,180
0,parcerita1,2020.01.01,parcerita1,mirapakaya,1-0,B54,Sicilian-Defense-Open-,2104,2140,180
0,parcerita1,2020.01.03,didip2,parcerita1,1-0,A16,English-Opening-Anglo-Indian-Queens-Knight-,2093,2100,180
0,parcerita1,2020.01.05,Wizard21,parcerita1,0-1,B43,Sicilian-Defense-Kan-Knight-,2091,2108,180
...,...,...,...,...,...,...,...,...,...,...
0,parcerita1,2020.12.09,magicianul666,parcerita1,0-1,B23,Sicilian-Defense-Closed-,2050,2114,180
0,parcerita1,2020.12.09,Tanov78,parcerita1,0-1,A16,English-Opening-Anglo-Indian-Queens-Knight-,2130,2123,180
0,parcerita1,2020.12.09,amano1,parcerita1,0-1,B43,Sicilian-Defense-Kan-Knight-,2146,2132,180
0,parcerita1,2020.12.09,parcerita1,Blitz-rider,0-1,B12,Caro-Kann-Defense-Modern-,2124,2136,180


In [46]:
# df_osk = get_player_games_yearly('leela100cores', 2020)
# df_osk